In [ ]:
from pdf2image import convert_from_path

#!apt-get install tesseract-ocr-deu
import os
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'

import pytesseract
import fitz  # PyMuPDF
import openai
import pandas as pd
#from google.colab import files
import time  # import time module for pausing
import json
import os

In [18]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_images(pdf_path):
    images = convert_from_path(pdf_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='deu')  # Verwende 'deu' für deutsche Texte
    return text

def extract_data_from_text(text, model="gpt-3.5-turbo", max_text_size=15000):
    text = text[:max_text_size]
    prompt = '''Du bekommst gleich einen String, der mit OCR aus einer Rechnung extrahiert wurde. Es handelt sich um eine einzelne Rechnung, die auch aus mehreren Seiten bestehen kann. Extrahiere die folgenden Daten aus diesem Text und gib sie im JSON-Format für die gesamte Rechnung aus. Gebe nur ein einzelnes JSON-Objekt aus. Hier ist eine Erklärung dazu, wie das JSON Objekt aussehen soll und was in die Felder rein soll: {
      "Vertragsnummer": "Hier soll die Vertragsnummer stehen. Diese ist in der Regel ein Integer, könnte aber in Ausnahmefällen auch andere Zeichen enthalten",
      "Adresse Verbrauchsstelle": "Das hier ist die Adresse der Verbrauchsstelle. Verwechsele sie nicht mit der Rechnungsadresse! Sie besteht aus Straße, Hausnummer, Postleitzahl und Ort",
      "Verbrauchte Menge": "Hier soll die verbrauchte Menge in Kilowattstunden (kWh) stehen. Wenn die Menge in Megawattstungen (MWh) angegeben ist, multipliziere mal 1000",
      "Start abgerechneter Zeitraum": "Gebe hier den Anfang des Abgerechneten Zeitraums als JJJJ-MM-TT an",
      "Ende abgerechneter Zeitraum": "Gebe hier das Ende des Abgerechneten Zeitraums als JJJJ-MM-TT an",
      "Energieart": "Hier soll entweder Gas oder Strom stehen",
      "Fehler": "Gib hier an, ob du bestimmte Daten nicht gefunden hast. Gib dann True an, ansonsten False. Damit wollen wir fehlerhafte Daten filtern"
      }
      Bitte gebe nur den JSON-String aus. Achte darauf, dass nur ein JSON-Objekt ausgegeben wird. Wenn ein Wert nicht gefunden wird, dann gib "nicht gefunden" an. Wenn Daten doppelt gefunden werden, dann fasse die Ergebnisse zusammen und gib nur den besten Treffer aus.'''
    # client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    client = openai.OpenAI(api_key='sk-proj-Ue2xY2FVsfuh1aNNTCg9T3BlbkFJQZgtifQyraq93mqfkMf3')
    response = client.chat.completions.create(
        model = model,
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": text}],
        max_tokens=150
    )

    # response = openai.ChatCompletion.create(
    #     model="gpt-3.5-turbo",
    #     messages=[{"role": "system", "content": prompt},
    #               {"role": "user", "content": text}],
    #     max_tokens=150
    # )
    extracted_data = response.choices[0].message['content'].strip()
    return extracted_data  # Remove any extra newlines at the end

def categorize_invoice(pdf_path):
    keywords_electricity = ["Strom", "kWh", "Netzbetreiber", "Energie"]
    keywords_gas = ["Gas", "m³", "Heizkosten", "Gasanbieter", "therm", "brennwert", "kWh/m³", "Gasverbrauch", "Gasrechnung"]

    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

    text = text.lower()

    electricity_score = sum(keyword.lower() in text for keyword in keywords_electricity)
    gas_score = sum(keyword.lower() in text for keyword in keywords_gas)

    if electricity_score > gas_score:
        return "Strom"
    elif gas_score > electricity_score:
        return "Gas"
    else:
        return "Nicht eindeutig"

def process_pdfs(file_list, model):
    extracted_data = []

    for file_path in file_list:
        #with open(file_path, 'wb') as f:
        #    f.write(file_content)

# KEIN SCHÖNER TEST. TODO: bessere Abfrage, ob erfolgreich 
        text = extract_text_from_pdf(file_path, model)
        if not text.strip():
            text = extract_text_from_images(file_path)  # Verwende OCR, wenn kein Text gefunden wurde
        data = extract_data_from_text(text)
        print(f"Extracted data from text for {file_name}:\n{data}\n")  # Debugging-Ausgabe
        extracted_data.append(data)

        time.sleep(5)  # Pause for 2 seconds between requests to avoid rate limiting

    return extracted_data



def extract_dict_from_response(response):
    response = response.split('{',1)[1].split('}',1)[0]
    response = '{' + response + '}'
    return json.loads(response)

def create_excel_file(data):
    data_dict = {'Vertragsnummer': [], 'Adresse Verbrauchsstelle': [], 'Verbrauchte Menge': [], 'Start abgerechneter Zeitraum': [], 'Ende abgerechneter Zeitraum':[], 'Energieart': [], 'Fehler':[]}

    for entry in data:
        temp_dict = extract_dict_from_response(entry)
        print(temp_dict)
        data_dict['Vertragsnummer'].append(temp_dict['Vertragsnummer'])
        data_dict['Adresse Verbrauchsstelle'].append(temp_dict['Adresse Verbrauchsstelle'])
        data_dict['Verbrauchte Menge'].append(temp_dict['Verbrauchte Menge'])
        data_dict['Start abgerechneter Zeitraum'].append(temp_dict['Start abgerechneter Zeitraum'])
        data_dict['Ende abgerechneter Zeitraum'].append(temp_dict['Ende abgerechneter Zeitraum'])
        data_dict['Energieart'].append(temp_dict['Energieart'])
        data_dict['Fehler'].append(temp_dict['Fehler'])

    df = pd.DataFrame(data_dict)
    display(df)
    output_file = '/content/extracted_data.xlsx'
    df.to_excel(output_file, index=False)
    return output_file

In [13]:


# set root directory for data
root_dir = "../data" # adjust to name of your root directory
# walk over root directory for data and get all csv files with their path and put them into a dataframe
path_list = []
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.pdf'):
            path_list.append(os.path.join(root, file))

print(path_list)

['../data\\R061023000112.pdf', '../data\\R090124000157.pdf', '../data\\R160623000581.pdf', '../data\\R160623000587.pdf', '../data\\R160623000593.pdf', '../data\\R180124000120.pdf', '../data\\R191223000248.pdf', '../data\\R211223000394.pdf', '../data\\R220124000415.pdf', '../data\\R220124000463.pdf', '../data\\R221223000164.pdf', '../data\\R271123000363.pdf', '../data\\R271123000366.pdf']


In [19]:
extract_text_from_pdf('../data\\R061023000112.pdf')

'IVU_40_SVK_Rechnung_enwg_EM V. 04 min. Schema_Version: 35\nEASY_NEUERKUNDE\nEBE_Belegart:Restforderung\nEZR_Zeitraum:01.08.2023 - 31.08.2023\nEFA_Fälligkeit:19.10.2023\nEVA_Verbrauchsstelle:Berufsbildungswerk Neumünster, Am Hohrkamp 54, D 24537 Neumünster\nERA_Rechnungsanschrift:Am Hohrkamp 54, D 24537 Neumünster\nERD_Rechnungsdatum:05.10.2023\nEBT_Betrag:3.287,05\nENA_NameRE:Berufsbildungswerk Neumünster  \nEKZ_Kundennummer2:\nEKN_Kundennummer:1201940779\nERN_Rechnungsnummer:000006585525\nEKM_KonzernMandant:S20\nEPZ_Prozess:563286\nEKG_Kundengruppe:2\nEZU_Unbundling:1\nELI_Listentitel:Rechnung\nStadtwerke Itzehoe GmbH • Postfach 19 55 • 25509 Itzehoe\nKunde: Berufsbildungswerk Neumünster  \nWilkenDMSId: W29787940\nFirma\nBerufsbildungswerk Neumünster\nAm Hohrkamp 54 \nD 24537 Neumünster\nRechnung\nAnsprechpartner:\nService Sondervertragskunden\nTelefon:\n04124 936-225\nTelefax:\n04124 936-130\nE-Mail:\ninfo@stadtwerke-itzehoe.de\nInternet:\nwww.stadtwerke-itzehoe.de\nKundennummer:\n1

In [14]:
# Daten aus PDF-Dateien extrahieren
extracted_data = process_pdfs(path_list)

TypeError: extract_data_from_text() missing 1 required positional argument: 'text'

In [20]:
extract_data_from_text(extract_text_from_pdf('../data\\R061023000112.pdf'))

TypeError: 'ChatCompletionMessage' object is not subscriptable